In [1]:
import os
import sys
sys.path.append(os.getcwd())

import seaborn as sns
import numpy as np
import cvxpy as cp
import time
from tqdm import tqdm
import ot
from scipy.special import factorial, gamma

import cv2
import torchvision
import torchvision.transforms as transforms

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
plt.rcParams["font.family"] = "serif"
plt.rcParams["figure.dpi"] = 200
plt.rcParams["font.size"] = 20
plt.rcParams["legend.fontsize"] = 14
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 7.0
plt.rcParams["xtick.minor.size"] = 5.0
plt.rcParams["ytick.major.size"] = 7.0
plt.rcParams["ytick.minor.size"] = 5.0
plt.rcParams["axes.linewidth"] = 1.5
plt.rcParams["legend.handlelength"] = 2.0
minor_locator = AutoMinorLocator(4)

Numerical experiment trên CIFAR10, đặt hypothesis, thí nghiệm, và ghi lại kết quả trên Overleaf
- **Xử Lý Trước Hình Ảnh**:
    - **Original Image**: Bắt đầu với các hình ảnh CIFAR-10, mỗi ảnh có kích thước $32 \times 32 \times 3$ (định dạng RGB).
    - **Grayscale Conversion**: Chuyển đổi các hình ảnh từ RGB sang ảnh grayscale.
    - **Downscaling**: Giảm kích thước của hình ảnh từ $32 \times 32$ xuống $10 \times 10$.
    - **Histogram Generation**: Làm phẳng ảnh grayscale đã giảm kích thước $10 \times 10$ thành một histogram với 100 bins.

- **Điều Chỉnh Histogram**:
    - **Zero-Avoidance**: Thêm một giá trị nhỏ $10^{-6}$ vào mỗi bin trong histogram để tránh các giá trị bằng 0.
    - **Normalization**:
        - Đối với mỗi cặp histogram (marginals), chia từng bin cho tổng khối lượng lớn nhất giữa hai histogram.
        - Điều này dẫn đến một marginal có tổng khối lượng bằng 1, trong khi marginal còn lại có thể có tổng khối lượng nhỏ hơn 1.

- **Tính Toán Ma Trận Cost**:
    - **Squared Euclidean Distance**: Sử dụng khoảng cách Euclidean bình phương giữa các vị trí pixel làm metric chi phí $C$ cho việc vận chuyển.
    - **Normalization**: Chuẩn hóa ma trận chi phí sao cho $\|\mathbf{C}\|_{\text{max}} = 1$.

- **Thiết Lập Bài Toán Vận Chuyển**:
    - **Transport Mass**: Đặt tổng khối lượng vận chuyển bằng 0.8 lần tổng khối lượng nhỏ nhất giữa hai marginals.

### a) Helper function

In [2]:
def measure_sparsity(solution_matrix, threshold=1e-10):
    """
    Hàm này đo lường sparsity của một ma trận, xem xét các phần tử có giá trị tuyệt đối
    nhỏ hơn ngưỡng đã chỉ định là bằng 0.

    Sparsity được định nghĩa là tỷ lệ các phần tử được xem là bằng 0 trong ma trận.

    Tham số:
    solution_matrix (np.ndarray): Ma trận cần đo lường sparsity.
    threshold (float): Ngưỡng dưới đó các phần tử được coi là bằng 0 (mặc định là 1e-10).

    Trả về:
    float: Độ sparsity của ma trận, dưới dạng giá trị từ 0 đến 1.
    """
    # Đếm số lượng phần tử được coi là bằng 0 (nhỏ hơn ngưỡng)
    zero_elements = np.sum(np.abs(solution_matrix) < threshold)

    # Tính tổng số phần tử trong ma trận
    total_elements = solution_matrix.size

    # Tính độ sparsity dưới dạng tỷ lệ của các phần tử được coi là bằng 0
    sparsity = zero_elements / total_elements

    return sparsity

In [3]:
# Hàm giải bài toán Optimal Transport (OT) không có regularizer
def sinkhornfix(a, b, C, epsilon, max_iter=1000, tol=1e-9, verbose=False):
    n, m = C.shape
    K = np.exp(-C / epsilon)
    
    u = np.ones(n)
    v = np.ones(m)
    
    for i in range(max_iter):
        u_prev, v_prev = u.copy(), v.copy()
        
        # Update u and v using the Sinkhorn iteration
        u = a / (np.dot(K, v) + 1e-12)  # Adding a small constant to avoid division by zero
        v = b / (np.dot(K.T, u) + 1e-12)

        if verbose:
            print("Iteration:", i)
            print("u:", u)
            print("v:", v)
        
        # Check for convergence
        if np.linalg.norm(u - u_prev, 1) < tol and np.linalg.norm(v - v_prev, 1) < tol:
            break
    
    P = np.outer(u, v) * K
    return P

def sinkhorn_log_domain(a, b, C, epsilon, max_iter=1000, tol=1e-9, verbose=False):
    n, m = C.shape
    
    # Instead of using K directly, work with its logarithm
    log_K = -C / epsilon
    
    # Initialize log_u and log_v
    log_u = np.zeros(n)
    log_v = np.zeros(m)
    
    for i in range(max_iter):
        log_u_prev, log_v_prev = log_u.copy(), log_v.copy()
        
        # Update log_u and log_v using log-domain Sinkhorn iterations
        log_u = np.log(a) - np.log(np.exp(log_K + log_v).sum(axis=1) + 1e-12)
        log_v = np.log(b) - np.log(np.exp(log_K.T + log_u).sum(axis=1) + 1e-12)

        if verbose:
            print("Iteration:", i)
            print("log_u:", log_u)
            print("log_v:", log_v)
        
        # Check for convergence
        if np.linalg.norm(log_u - log_u_prev, 1) < tol and np.linalg.norm(log_v - log_v_prev, 1) < tol:
            break
    
    # Compute the transport plan in the original space
    u = np.exp(log_u)
    v = np.exp(log_v)
    P = np.outer(u, v) * np.exp(log_K)
    
    return P

def solve_ot(source_hist, target_hist, C):
    # Khai báo biến kế hoạch vận chuyển
    X = cp.Variable(C.shape, nonneg=True)

    # Định nghĩa các ràng buộc cho bài toán POT
    constraints = [
        cp.sum(X, axis=1) == source_hist.reshape(-1, 1).flatten(),
        cp.sum(X, axis=0) == target_hist.reshape(-1, 1).flatten(),
    ]
    # Định nghĩa hàm mục tiêu chỉ với cost matrix C mà không có regularizer
    objective = cp.Minimize(cp.sum(cp.multiply(C, X)))

    # Định nghĩa và giải bài toán
    prob = cp.Problem(objective, constraints)
    prob.solve()

    return X.value

# Hàm giải bài toán Optimal Transport (OT) có regularizer
def solve_ot_quadratic_regularization(source_hist, target_hist, C, epsilon):
    # Khai báo biến kế hoạch vận chuyển
    X = cp.Variable(C.shape, nonneg=True)

    # Định nghĩa các ràng buộc cho bài toán POT
    constraints = [
        cp.sum(X, axis=1) == source_hist.reshape(-1, 1).flatten(),
        cp.sum(X, axis=0) == target_hist.reshape(-1, 1).flatten(),
    ]

    # Định nghĩa hàm mục tiêu chỉ với cost matrix C mà không có regularizer
    objective = cp.Minimize(cp.sum(cp.multiply(C, X))+(epsilon / 2) * cp.norm(X, "fro") ** 2)

    # Định nghĩa và giải bài toán
    prob = cp.Problem(objective, constraints)
    prob.solve()

    return X.value

# Hàm giải bài toán Optimal Transport (OT) không có regularizer
def solve_pot_entropic_regularization(source_hist, target_hist, C, s, epsilon):
    # Khai báo biến kế hoạch vận chuyển
    X = cp.Variable(C.shape, nonneg=True)

    # Định nghĩa các ràng buộc cho bài toán POT
    constraints = [
        cp.sum(X, axis=1) <= source_hist.reshape(-1, 1).flatten(),
        cp.sum(X, axis=0) <= target_hist.reshape(-1, 1).flatten(),
        cp.sum(X) == s,
    ]

    # Định nghĩa hàm mục tiêu chỉ với cost matrix C mà không có regularizer
    def entropy(X):
        tau=1e-10
        return cp.sum(cp.entr(X + tau))  # Use the entr function which is DCP-compliant

    # Define the objective function
    objective = cp.Minimize(cp.sum(cp.multiply(C, X)) - (epsilon/2) * entropy(X))

    # Định nghĩa và giải bài toán
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.SCS, max_iters=3000, eps=1e-8, verbose=True)

    return X.value

def solve_pot_entropic_kl_regularization(source_hist, target_hist, C, s, epsilon):
    # Khai báo biến kế hoạch vận chuyển
    X = cp.Variable(C.shape, nonneg=True)

    # Định nghĩa các ràng buộc cho bài toán POT
    constraints = [
        cp.sum(X, axis=1) <= source_hist.reshape(-1, 1).flatten(),
        cp.sum(X, axis=0) <= target_hist.reshape(-1, 1).flatten(),
        cp.sum(X) == s,
    ]

    # Define the cost term: sum(C * X)
    cost_term = cp.sum(cp.multiply(C, X))

    # Define the entropic regularization term: sum(X_ij * log(X_ij) - X_ij)
    # Using cp.kl_div(X_ij, 1) which computes X_ij * log(X_ij) - X_ij for each element
    entropy_term = cp.sum(cp.kl_div(X, np.ones((n, m))))

    # Define the objective function: cost term + epsilon * entropy term
    objective = cp.Minimize(cost_term + epsilon * entropy_term)


    # Định nghĩa và giải bài toán
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.SCS, max_iters=5000)

    return X.value

def solve_ot_entropic_regularization(source_hist, target_hist, C, epsilon):
    # Khai báo biến kế hoạch vận chuyển
    X = cp.Variable(C.shape, nonneg=True)

    # Định nghĩa các ràng buộc cho bài toán POT
    constraints = [
        cp.sum(X, axis=1) == source_hist.reshape(-1, 1).flatten(),
        cp.sum(X, axis=0) == target_hist.reshape(-1, 1).flatten(),
    ]

    # Định nghĩa hàm mục tiêu chỉ với cost matrix C mà không có regularizer
    def entropy(X):
        tau=1e-10
        return cp.sum(cp.entr(X + tau))  # Use the entr function which is DCP-compliant

    # Define the objective function
    objective = cp.Minimize(cp.sum(cp.multiply(C, X)) - (epsilon/2) * entropy(X))

    # Định nghĩa và giải bài toán
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.SCS)

    return X.value

# Hàm giải POT với quadratic regularizer
def solve_pot_quadratic_regularization(source_hist, target_hist, C, epsilon, s):
    # Khai báo biến kế hoạch vận chuyển
    X = cp.Variable(C.shape, nonneg=True)

    # Định nghĩa các ràng buộc cho bài toán POT
    constraints = [
        cp.sum(X, axis=1) <= source_hist.reshape(-1, 1).flatten(),
        cp.sum(X, axis=0) <= target_hist.reshape(-1, 1).flatten(),
        cp.sum(X) == s,
    ]

    # Định nghĩa hàm mục tiêu với regularization quadratic
    objective = cp.Minimize(cp.trace(C.T @ X) + (epsilon / 2) * cp.norm(X, "fro") ** 2)

    # Định nghĩa và giải bài toán
    prob = cp.Problem(objective, constraints)
    prob.solve()

    return X.value


def solve_pot(source_hist, target_hist, C, s):
    # Khai báo biến kế hoạch vận chuyển
    X = cp.Variable(C.shape, nonneg=True)

    # Định nghĩa các ràng buộc cho bài toán POT
    constraints = [
        cp.sum(X, axis=1) <= source_hist.reshape(-1, 1).flatten(),
        cp.sum(X, axis=0) <= target_hist.reshape(-1, 1).flatten(),
        cp.sum(X) == s,
    ]

    # Định nghĩa hàm mục tiêu chỉ với cost matrix C mà không có regularizer
    objective = cp.Minimize(cp.sum(cp.multiply(C, X)))


    # Định nghĩa và giải bài toán
    prob = cp.Problem(objective, constraints)
    prob.solve()

    return X.value

def round_heatmap(method_matrices, epsilon, lambda_param, threshold=1e-10):
    """
    Plots heatmaps for the provided method matrices (solutions) and includes epsilon in the plot title.

    :param threshold: The threshold value to discretize the solution matrices.
    :param epsilon: The epsilon value to display in the plot title.
    :param method_matrices: A dictionary where keys are method names and values are the corresponding solution matrices.
    """
    # Discretize all input matrices (round them based on threshold)
    rounded_matrices = {method: np.where(np.abs(matrix) < threshold, 0, 1) for method, matrix in method_matrices.items()}

    num_matrices = len(rounded_matrices)
    max_columns = min(3, num_matrices)  # Set the number of columns based on the number of matrices
    num_rows = (num_matrices + max_columns - 1) // max_columns  # Calculate the number of rows needed

    fig, axes = plt.subplots(num_rows, max_columns, figsize=(6 * max_columns, 6 * num_rows))
    axes = axes.flatten() if num_matrices > 1 else [axes]  # Flatten axes for easy iteration if more than one matrix

    for idx, (method_name, rounded_matrix) in enumerate(rounded_matrices.items()):
        ax = axes[idx]

        # Draw the heatmap for each method
        sns.heatmap(rounded_matrix, cmap=sns.color_palette(["white", "black"], as_cmap=True), ax=ax, cbar=False)

        # Set title and format the plot
        ax.set_title(f"Transport Plan {method_name}\nEpsilon: {epsilon}, Lambda: {lambda_param} \n Sparsity: {np.round(measure_sparsity(rounded_matrix), 4)}", fontsize=16)
        ax.set_xticks([])  # Remove x-axis ticks
        ax.set_yticks([])  # Remove y-axis ticks

        # Add a box around the heatmap
        for _, spine in ax.spines.items():
            spine.set_visible(True)
            spine.set_linewidth(1.5)
            spine.set_color('black')

    # Hide unused axes
    for idx in range(len(rounded_matrices), len(axes)):
        axes[idx].axis('off')

    plt.tight_layout()
    # plt.savefig(f"Lambdar_{lambda_param}.png")  # Save the figure if necessary

    return fig


In [4]:
# Hàm tạo phân phối Mixed Gaussian
def generate_mixed_gaussian(n_samples, n_bins, mu1, sigma1, mu2, sigma2):
    bernoulli = np.random.binomial(n=1, p=0.5, size=n_samples)
    gaussian1 = np.random.normal(mu1, sigma1, n_samples)
    gaussian2 = np.random.normal(mu2, sigma2, n_samples)
    dist = gaussian1 * bernoulli + gaussian2 * (1 - bernoulli)

    x = np.linspace(min(dist), max(dist), n_samples)
    p, edges = np.histogram(dist, bins=n_bins, density=True)

    pdf = 0.5 * (
        1 / (sigma1 * np.sqrt(2 * np.pi)) * np.exp(-0.5 * ((x - mu1) / sigma1) ** 2)
    ) + 0.5 * (
        1 / (sigma2 * np.sqrt(2 * np.pi)) * np.exp(-0.5 * ((x - mu2) / sigma2) ** 2)
    )

    return dist, p, edges, x, pdf


# Hàm tạo phân phối Poisson
def generate_poisson(n_samples, n_bins, lambda_param):
    dist = np.random.poisson(lambda_param, n_samples)

    x = np.linspace(min(dist), max(dist), n_samples)
    p, edges = np.histogram(dist, bins=n_bins, density=True)

    pdf = np.exp(-lambda_param) * np.power(lambda_param, x) / factorial(x)

    return dist, p, edges, x, pdf


# Hàm tạo phân phối Beta
def generate_beta(n_samples, n_bins, alpha, beta_param):
    dist = np.random.beta(alpha, beta_param, n_samples)

    p, edges = np.histogram(dist, bins=n_bins, density=True)

    x = np.linspace(min(dist), max(dist), n_samples)
    pdf = (x ** (alpha - 1) * (1 - x) ** (beta_param - 1)) / (
        gamma(alpha) * gamma(beta_param) / gamma(alpha + beta_param)
    )

    return dist, p, edges, x, pdf


# Hàm tạo phân phối Binomial
def generate_binomial(n_samples, n_bins, n, p):
    dist = np.random.binomial(n, p, n_samples)

    x = np.linspace(min(dist), max(dist), n_samples)
    p_dist, edges = np.histogram(dist, bins=n_bins, density=True)

    pdf = (
        (factorial(n) / (factorial(x) * factorial(n - x)))
        * (p**x)
        * ((1 - p) ** (n - x))
    )

    return dist, p_dist, edges, x, pdf


# Hàm tạo phân phối Gamma
def generate_gamma(n_samples, n_bins, shape, scale):
    dist = np.random.gamma(shape, scale, n_samples)

    x = np.linspace(min(dist), max(dist), n_samples)
    p, edges = np.histogram(dist, bins=n_bins, density=True)

    pdf = (x ** (shape - 1) * np.exp(-x / scale)) / (scale**shape * gamma(shape))

    return dist, p, edges, x, pdf

### c) Numerical experiments on Toy distributions

#### Experiment between distributions

In [5]:
# Define the fixed parameters for the experiment
n_bins = 100
n_samples = 100000
mu1, sigma1 = 1, 2
mu2, sigma2 = 10, 1.5
epsilon = 10
lambda_param = 0.99

# Define the distribution pairs
distribution_names = ["Mixed Gaussian", "Gamma", "Beta", "Poisson", "Binomial"]

# Store results
runtimes = {}
sparsities = {}

# Iterate through all possible pairs of distributions
for i, dist_a_name in enumerate(distribution_names):
    for j, dist_b_name in enumerate(distribution_names):
        if dist_a_name == dist_b_name:
            continue
        # Generate the first distribution based on the name
        if dist_a_name == "Mixed Gaussian":
            dist_a, p_a, edges_a, x_a, pdf_a = generate_mixed_gaussian(
                n_samples, n_bins, mu1, sigma1, mu2, sigma2
            )
        elif dist_a_name == "Gamma":
            shape_a, scale_a = 7, 1
            dist_a, p_a, edges_a, x_a, pdf_a = generate_gamma(
                n_samples, n_bins, shape_a, scale_a
            )
        elif dist_a_name == "Beta":
            alpha_a, beta_a = 2, 2
            dist_a, p_a, edges_a, x_a, pdf_a = generate_beta(
                n_samples, n_bins, alpha_a, beta_a
            )
        elif dist_a_name == "Poisson":
            lambda_a = 5
            dist_a, p_a, edges_a, x_a, pdf_a = generate_poisson(n_samples, n_bins, lambda_a)
        elif dist_a_name == "Binomial":
            n_a, p_binom_a = 10, 0.4
            dist_a, p_a, edges_a, x_a, pdf_a = generate_binomial(
                n_samples, n_bins, n_a, p_binom_a
            )

        # Generate the second distribution based on the current pair
        if dist_b_name == "Mixed Gaussian":
            dist_b, p_b, edges_b, x_b, pdf_b = generate_mixed_gaussian(
                n_samples, n_bins, mu1, sigma1, mu2, sigma2
            )
        elif dist_b_name == "Gamma":
            shape_b, scale_b = 7, 1
            dist_b, p_b, edges_b, x_b, pdf_b = generate_gamma(
                n_samples, n_bins, shape_b, scale_b
            )
        elif dist_b_name == "Beta":
            alpha_b, beta_b = 2, 2
            dist_b, p_b, edges_b, x_b, pdf_b = generate_beta(
                n_samples, n_bins, alpha_b, beta_b
            )
        elif dist_b_name == "Poisson":
            lambda_b = 5
            dist_b, p_b, edges_b, x_b, pdf_b = generate_poisson(n_samples, n_bins, lambda_b)
        elif dist_b_name == "Binomial":
            n_b, p_binom_b = 10, 0.4
            dist_b, p_b, edges_b, x_b, pdf_b = generate_binomial(
                n_samples, n_bins, n_b, p_binom_b
            )

        # Normalize distributions
        p_a = p_a / np.sum(p_a)
        p_b = p_b / np.sum(p_b)
        s = lambda_param * min(p_a.sum(), p_b.sum())

        # Compute cost matrix using L2 norm
        C = (edges_a[:-1].reshape(-1, 1) - edges_b[:-1].reshape(1, -1)) ** 2
        C = C / np.max(C)

        # Solve the POT problem using QPOT
        start_time = time.time()
        X_qpot = solve_pot_quadratic_regularization(p_a, p_b, C, epsilon, s)
        runtime_qpot = time.time() - start_time

        # Measure sparsity of the solution matrix
        sparsity_qpot = measure_sparsity(X_qpot)

        # Solve the POT problem using EPOT
        start_time = time.time()
        X_epot = solve_pot_entropic_regularization(p_a, p_b, C, epsilon, s)
        runtime_epot = time.time() - start_time

        # Measure sparsity of the solution matrix
        sparsity_epot = measure_sparsity(X_epot)

        # Store results
        if np.abs(sparsity_qpot - sparsity_epot) > 0.1:
            runtimes[(dist_a_name, dist_b_name)] = [runtime_qpot, runtime_epot]
            sparsities[(dist_a_name, dist_b_name)] = [sparsity_qpot, sparsity_epot]


figure_path = 'toy_distribution_experiment'
os.makedirs(figure_path, exist_ok=True)
# Extract data from the sparsities dictionary
dist_pairs = list(sparsities.keys())
qpot_sparsities = [sparsities[pair][0] for pair in dist_pairs]  # Sparsities for QPOT
epot_sparsities = [sparsities[pair][1] for pair in dist_pairs]  # Sparsities for EPOT

# Define the positions for the bars
x = np.arange(len(dist_pairs))  # Number of distribution pairs
width = 0.35  # Width of the bars

# Create the plot
fig, ax = plt.subplots(figsize=(10, 6))

# Plot bars for QPOT and EPOT sparsities
bars_qpot = ax.bar(x - width/2, qpot_sparsities, width, label='QPOT Sparsity', color='b')
bars_epot = ax.bar(x + width/2, epot_sparsities, width, label='EPOT Sparsity', color='r')

# Add labels, title, and custom ticks
ax.set_xlabel('Distribution Pairs', fontsize=14)
ax.set_ylabel('Sparsity', fontsize=14)
ax.set_title('Comparison of Sparsity: QPOT vs EPOT', fontsize=16)
ax.set_xticks(x)
ax.set_xticklabels([f'{pair[0]} vs {pair[1]}' for pair in dist_pairs], rotation=45, ha='right')
ax.legend()

# Display plot
plt.tight_layout()
plt.savefig(os.path.join(figure_path, 'sparsity_pairs of distribution.png'))
plt.show()

qpot_runtimes = [runtimes[pair][0] for pair in dist_pairs]  # Sparsities for QPOT
epot_runtimes = [runtimes[pair][1] for pair in dist_pairs]  # Sparsities for EPOT

# Define the positions for the bars
x = np.arange(len(dist_pairs))  # Number of distribution pairs
width = 0.35  # Width of the bars

# Create the plot
fig, ax = plt.subplots(figsize=(10, 6))

# Plot bars for QPOT and EPOT sparsities
bars_qpot = ax.bar(x - width/2, qpot_runtimes, width, label='QPOT Sparsity', color='b')
bars_epot = ax.bar(x + width/2, epot_runtimes, width, label='EPOT Sparsity', color='r')

# Add labels, title, and custom ticks
ax.set_xlabel('Distribution Pairs', fontsize=14)
ax.set_ylabel('Runtime', fontsize=14)
ax.set_title('Comparison of Runtime: QPOT vs EPOT', fontsize=16)
ax.set_xticks(x)
ax.set_xticklabels([f'{pair[0]} vs {pair[1]}' for pair in dist_pairs], rotation=45, ha='right')
ax.legend()

# Display plot
plt.tight_layout()
plt.savefig(os.path.join(figure_path, 'runtime_pairs of distribution.png'))
plt.show()

                                     CVXPY                                     
                                     v1.5.2                                    
(CVXPY) Sep 27 07:11:47 PM: Your problem has 10000 variables, 201 constraints, and 0 parameters.
(CVXPY) Sep 27 07:11:47 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 27 07:11:47 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 27 07:11:47 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Sep 27 07:11:47 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 27 07:11:47 PM: Compiling problem (target solver=SCS).


TypeError: bad operand type for abs(): 'NoneType'

#### Experiment on multiple epsilon

In [ ]:
# Define the fixed parameters for the experiment
n_bins = 100
n_samples = 100000
mu1, sigma1 = 1, 2
mu2, sigma2 = 10, 1.5
epsilon = 10
lambda_param = 0.99

epsilon_list = [2**(-i) for i in range(0, 51)]

dist_a, p_a, edges_a, x_a, pdf_a = generate_mixed_gaussian(
                n_samples, n_bins, mu1, sigma1, mu2, sigma2
            )

dist_b, p_b, edges_b, x_b, pdf_b = generate_gamma(
                n_samples, n_bins, 7, 1
            )

# Normalize distributions
p_a = p_a / np.sum(p_a)
p_b = p_b / np.sum(p_b)
s = lambda_param * min(p_a.sum(), p_b.sum())

# Compute cost matrix using L2 norm
C = (edges_a[:-1].reshape(-1, 1) - edges_b[:-1].reshape(1, -1)) ** 2
C = C / np.max(C)

sparsities = {'QPOT' : [], 'EPOT': []}
runtimes = {'QPOT' : [], 'EPOT': []}
for epsilon in epsilon_list:
    # Solve the POT problem using QPOT
    start_time = time.time()
    X_qpot = solve_pot_quadratic_regularization(p_a, p_b, C, epsilon, s)
    runtime_qpot = time.time() - start_time

    # Measure sparsity of the solution matrix
    sparsity_qpot = measure_sparsity(X_qpot)
    sparsities['QPOT'].append(sparsity_qpot)
    runtimes['QPOT'].append(runtime_qpot)

    # Solve the POT problem using EPOT
    start_time = time.time()
    X_epot = solve_pot_entropic_regularization(p_a, p_b, C, epsilon, s)
    runtime_epot = time.time() - start_time

    # Measure sparsity of the solution matrix
    sparsity_epot = measure_sparsity(X_epot)
    sparsities['EPOT'].append(sparsity_epot)
    runtimes['EPOT'].append(runtime_epot)

# Create the plot
plt.figure(figsize=(15, 9))

# Plot with improved line styles and colors
plt.plot(x, sparsities["QPOT"], label="QPOT", color='#1f77b4', linewidth=1.5, linestyle='-')
plt.plot(x, sparsities["EPOT"], label='EPOT', color='r', linewidth=1.5, linestyle='--')
# plt.plot(x, sparsity_cifar["KLPOT"], label='KLPOT', color='blue', linewidth=1.5, linestyle='-')
# plt.plot(x, sparsity_cifar["QOT"], label='QOT', color='#ff7f0e', linewidth=1.5, linestyle='--')
# plt.plot(x, sparsity_cifar["EOT"], label='EOT', color='#2ca02c', linewidth=1.5, linestyle='-.')
# plt.plot(x, sparsity_cifar["EOTL"], label='EOT Library', color='m', linewidth=1.5, linestyle='-.')
# plt.plot(x, sparsity_cifar["LEOT"], label='Log EOT', color='r', linewidth=1.5, linestyle='-.')

# Draw horizontal lines for OT and POT
#plt.hlines(y=sparsity_cifar["OT"], xmin=2**(0), xmax=2**(-51), color='k', linewidth=1.5, linestyle='--', label='OT')
#plt.hlines(y=sparsity_cifar["POT"], xmin=2**(0), xmax=2**(-51), color='k', linewidth=1.5, linestyle='-', label='POT')

# Set title and labels with improved font settings
plt.title('Sparsity of numerical Experiment on Mixed Gaussian and Gamma with λ = ' + str(lambda_param), fontsize=25, fontweight='bold')
plt.xlabel(r'$\varepsilon$ coefficient', fontsize=22, fontdict={'family': 'serif', 'weight': 'bold'})
plt.ylabel('Sparsity', fontsize=22, fontdict={'family': 'serif', 'weight': 'bold'})

# Add a grid with a specific style
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(rotation=-45)

# Adjust tick parameters
plt.xscale("log")
plt.legend(fontsize=16)
plt.gca().invert_xaxis() 

# Display the plot
plt.tight_layout()
plt.savefig(os.path.join(figure_path, f'sparsity_multiple_epsilon_lambda{lambda_param}.png'))

# Create the plot
plt.figure(figsize=(15, 9))

# Plot with improved line styles and colors
plt.plot(x, runtimes["QPOT"], label="QPOT", color='#1f77b4', linewidth=1.5, linestyle='-')
plt.plot(x, runtimes["EPOT"], label='EPOT', color='r', linewidth=1.5, linestyle='--')
# plt.plot(x, sparsity_cifar["KLPOT"], label='KLPOT', color='blue', linewidth=1.5, linestyle='-')
# plt.plot(x, sparsity_cifar["QOT"], label='QOT', color='#ff7f0e', linewidth=1.5, linestyle='--')
# plt.plot(x, sparsity_cifar["EOT"], label='EOT', color='#2ca02c', linewidth=1.5, linestyle='-.')
# plt.plot(x, sparsity_cifar["EOTL"], label='EOT Library', color='m', linewidth=1.5, linestyle='-.')
# plt.plot(x, sparsity_cifar["LEOT"], label='Log EOT', color='r', linewidth=1.5, linestyle='-.')

# Draw horizontal lines for OT and POT
#plt.hlines(y=sparsity_cifar["OT"], xmin=2**(0), xmax=2**(-51), color='k', linewidth=1.5, linestyle='--', label='OT')
#plt.hlines(y=sparsity_cifar["POT"], xmin=2**(0), xmax=2**(-51), color='k', linewidth=1.5, linestyle='-', label='POT')

# Set title and labels with improved font settings
plt.title('Runtimes of numerical Experiment on Mixed Gaussian and Gamma with λ = ' + str(lambda_param), fontsize=25, fontweight='bold')
plt.xlabel(r'$\varepsilon$ coefficient', fontsize=22, fontdict={'family': 'serif', 'weight': 'bold'})
plt.ylabel('Runtimes', fontsize=22, fontdict={'family': 'serif', 'weight': 'bold'})

# Add a grid with a specific style
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(rotation=-45)

# Adjust tick parameters
plt.xscale("log")
plt.legend(fontsize=16)
plt.gca().invert_xaxis() 

# Display the plot
plt.tight_layout()
plt.savefig(os.path.join(figure_path, f'sparsity_multiple_epsilon_lambda{lambda_param}.png'))

#### Experiment on multiple lambda

In [ ]:
n_bins = 100
n_samples = 100000
mu1, sigma1 = 1, 2
mu2, sigma2 = 10, 1.5
epsilon = 10

runtimes = {}
sparsities = {}

lambda_list = [0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]

dist_a, p_a, edges_a, x_a, pdf_a = generate_mixed_gaussian(
                n_samples, n_bins, mu1, sigma1, mu2, sigma2
            )

dist_b, p_b, edges_b, x_b, pdf_b = generate_gamma(
                n_samples, n_bins, 7, 1
            )

# Normalize distributions
p_a = p_a / np.sum(p_a)
p_b = p_b / np.sum(p_b)

# Compute cost matrix using L2 norm
C = (edges_a[:-1].reshape(-1, 1) - edges_b[:-1].reshape(1, -1)) ** 2
C = C / np.max(C)

sparsities = {'QPOT' : [], 'EPOT': []}
runtimes = {'QPOT' : [], 'EPOT': []}
for lambda_param in lambda_list:
    s = lambda_param * min(p_a.sum(), p_b.sum())
    # Solve the POT problem using QPOT
    start_time = time.time()
    X_qpot = solve_pot_quadratic_regularization(p_a, p_b, C, epsilon, s)
    runtime_qpot = time.time() - start_time

    # Measure sparsity of the solution matrix
    sparsity_qpot = measure_sparsity(X_qpot)

    # Solve the POT problem using EPOT
    start_time = time.time()
    X_epot = solve_pot_entropic_regularization(p_a, p_b, C, epsilon, s)
    runtime_epot = time.time() - start_time

    # Measure sparsity of the solution matrix
    sparsity_epot = measure_sparsity(X_epot)
    
    sparsities[lambda_param] = (sparsity_qpot, sparsity_epot)
    runtimes[lambda_param] = (runtime_qpot, runtime_epot)
    
# Extract data from the sparsities dictionary
qpot_sparsities = [sparsities[lambda_param][0] for lambda_param in lambda_list]  # Sparsities for QPOT
epot_sparsities = [sparsities[lambda_param][1] for lambda_param in lambda_list]  # Sparsities for EPOT

# Define the positions for the bars
x = np.arange(len(lambda_list))  # Number of distribution pairs
width = 0.35  # Width of the bars

# Create the plot
fig, ax = plt.subplots(figsize=(10, 6))

# Plot bars for QPOT and EPOT sparsities
bars_qpot = ax.bar(x - width/2, qpot_sparsities, width, label='QPOT Sparsity', color='b')
bars_epot = ax.bar(x + width/2, epot_sparsities, width, label='EPOT Sparsity', color='r')

# Add labels, title, and custom ticks
ax.set_xlabel('Lambda', fontsize=14)
ax.set_ylabel('Sparsity', fontsize=14)
ax.set_title('Comparison of Sparsity on multiple Lambda: QPOT vs EPOT', fontsize=16)
ax.set_xticks(x)
ax.set_xticklabels([f'{lambda_param}' for lambda_param in lambda_list], rotation=45, ha='right')
ax.legend()

# Display plot
plt.tight_layout()
plt.savefig(os.path.join(figure_path, f'sparsity_multiple_lambda_lambda{lambda_param}.png'))
plt.show()

qpot_runtimes = [sparsities[lambda_param][0] for lambda_param in lambda_list]  # Sparsities for QPOT
epot_runtimes = [sparsities[lambda_param][1] for lambda_param in lambda_list]  # Sparsities for EPOT

# Define the positions for the bars
x = np.arange(len(lambda_list))  # Number of distribution pairs
width = 0.35  # Width of the bars

# Create the plot
fig, ax = plt.subplots(figsize=(10, 6))

# Plot bars for QPOT and EPOT sparsities
bars_qpot = ax.bar(x - width/2, qpot_runtimes, width, label='QPOT Sparsity', color='b')
bars_epot = ax.bar(x + width/2, epot_runtimes, width, label='EPOT Sparsity', color='r')

# Add labels, title, and custom ticks
ax.set_xlabel('Lambda', fontsize=14)
ax.set_ylabel('Runtime', fontsize=14)
ax.set_title('Comparison of Runtime on multiple Lambda: QPOT vs EPOT', fontsize=16)
ax.set_xticks(x)
ax.set_xticklabels([f'{lambda_param}' for lambda_param in lambda_list], rotation=45, ha='right')
ax.legend()

# Display plot
plt.tight_layout()
plt.savefig(os.path.join(figure_path, f'runtime_multiple_lambda_lambda{lambda_param}.png'))
plt.show()